In [1]:
import numpy as np
import pandas as pd
import pyspark
import pyspark.sql.functions as F
import pydataset
from pydataset import data
spark = pyspark.sql.SparkSession.builder.getOrCreate()
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 

import env 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/24 15:28:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/24 15:28:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Exercises
### Data Acquisition
### These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

### This exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.
## PART I.
### 1. Read the case, department, and source data into their own spark dataframes.

In [2]:
from env import host, username, password

def get_db_url(database, host=host, user=username, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'


In [3]:
#reading case data into spark dataframe
query = """SELECT * FROM cases"""
url = get_db_url("311_data")
case = pd.read_sql(query, url)
case = spark.createDataFrame(case)
case.show(4)

22/10/24 15:31:59 WARN TaskSetManager: Stage 0 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 15:32:03 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8

In [4]:
#reading dept data into spark dataframe
query = """SELECT * FROM dept"""
dept = pd.read_sql(query, url)
dept = spark.createDataFrame(dept)
dept.show(4)


+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 4 rows



In [5]:
#reading source data into spark dataframe
query = """SELECT * FROM source"""
url = get_db_url("311_data")
source_df = pd.read_sql(query, url)
source_df = spark.createDataFrame(source_df)
source_df.show(4)

+-----+---------+----------------+
|index|source_id| source_username|
+-----+---------+----------------+
|    0|   100137|Merlene Blodgett|
|    1|   103582|     Carmen Cura|
|    2|   106463| Richard Sanchez|
|    3|   119403|  Betty De Hoyos|
+-----+---------+----------------+
only showing top 4 rows



### 2. Let's see how writing to the local disk works in spark:
a) Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json.

b) Inspect your folder structure. What do you notice?

In [6]:
# Writing source dataframe into json and csv formats
source_df.write.json('source_json', mode = 'overwrite')
source_df.write.csv('source_csv', mode = 'overwrite')

### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [7]:
#inspecting data in my dataframes
case.show(1, vertical = True, truncate = False)

22/10/24 15:32:08 WARN TaskSetManager: Stage 6 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 15:32:13 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 6 (TID 20): Attempting to kill Python Worker
-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616                         
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 7820

In [8]:
# check the case dataframe
case.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: long (nullable = true)



In [9]:
#create case_opened_date column
fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", F.to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", F.to_timestamp("case_closed_date", fmt))
    .withColumn("SLA_due_date", F.to_timestamp("SLA_due_date", fmt))
)

case = (
    case.withColumn("case_closed", F.expr('case_closed == "YES"'))
    .withColumn("case_late", F.expr('case_late == "YES"'))
)

In [10]:
dept.show(1, vertical = True, truncate = False)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [11]:
# check the department dataframe
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [12]:
#dept_subject_to_SLA column looks like a boolean type.
dept.select('dept_subject_to_SLA').distinct().show()

+-------------------+
|dept_subject_to_SLA|
+-------------------+
|                YES|
|                 NO|
+-------------------+



## PART II.
### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [13]:
# Let's open the case with SQL
case.createOrReplaceTempView('case')

In [14]:
#using sql querry to check how old the latest (in terms of days past SLA) currently open issue
spark.sql('''
SELECT
    num_days_late
FROM case
WHERE case_closed = false
    AND case_late = true
ORDER BY num_days_late DESC
LIMIT 1;
''').show(vertical = True, truncate = False)

22/10/24 15:32:15 WARN TaskSetManager: Stage 11 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0--------------------
 num_days_late | 348.6458333 



In [15]:
#using sql querry to check how long the oldest (in terms of days since opened) currently opened issue been open
spark.sql('''
SELECT
    DATEDIFF(NOW(), case_opened_date) AS days_open
FROM case
WHERE case_closed = false
ORDER BY days_open DESC
LIMIT 1;
''').show(vertical = True)

22/10/24 15:32:19 WARN TaskSetManager: Stage 13 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0---------
 days_open | 2122 



### 2. How many Stray Animal cases are there?

In [16]:
#using sql querry to check how many stray animal cases are there
spark.sql('''
SELECT
    COUNT(*)
FROM case
WHERE service_request_type = 'Stray Animal';
''').show(vertical = True)

22/10/24 15:32:22 WARN TaskSetManager: Stage 15 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0---------
 count(1) | 26760 



### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [17]:
#using sql querry to check how many service requests are assigned to dept_div
spark.sql('''
SELECT
    COUNT(*)
FROM case
WHERE dept_division = 'Field Operations'
    AND service_request_type != 'Officer Standby';
''').show(vertical = True)

22/10/24 15:32:24 WARN TaskSetManager: Stage 18 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


-RECORD 0----------
 count(1) | 113902 



### 4. Convert the council_district column to a string column.

In [18]:
#converting council_district column to a string
case = case.withColumn('council_district', F.col('council_district').cast('string'))
case.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



### 5. Extract the year from the case_closed_date column.

In [19]:
#extract year from case closed date column
case.select(F.year('case_closed_date')).show(5)

22/10/24 15:32:26 WARN TaskSetManager: Stage 21 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 15:32:30 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 21 (TID 67): Attempting to kill Python Worker
+----------------------+
|year(case_closed_date)|
+----------------------+
|                  2018|
|                  2018|
|                  2018|
|                  2018|
|                  2018|
+----------------------+
only showing top 5 rows



### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [20]:
#converting num days late from days to hours late
case = case.withColumn('num_hours_late', F.expr('num_days_late * 24'))
case.show(1, vertical = True)

22/10/24 15:32:30 WARN TaskSetManager: Stage 22 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 15:32:34 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 22 (TID 68): Attempting to kill Python Worker
-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616         
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 num_hours_late       | -23964.210278399998  
only showing top 1 row



In [21]:
#converting num days late from days to hours late
case.select('num_days_late', 'num_hours_late').show(5)

22/10/24 15:32:34 WARN TaskSetManager: Stage 23 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 15:32:38 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 23 (TID 69): Attempting to kill Python Worker
+-------------+-------------------+
|num_days_late|     num_hours_late|
+-------------+-------------------+
| -998.5087616|-23964.210278399998|
| -2.012604167|      -48.302500008|
| -3.022337963|      -72.536111112|
| -15.01148148|      -360.27555552|
|  0.372164352|        8.931944448|
+-------------+-------------------+
only showing top 5 rows



### 7. Join the case data with the source and department data.

In [22]:
#checking schema for each dataframe before joining them all
#case data
case.printSchema()

root
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- num_hours_late: double (nullable = true)



In [23]:
#dept data 
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [24]:
#source_df data
source_df.printSchema()

root
 |-- index: long (nullable = true)
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [25]:
#function to join dept dataframe to dept_division and source_df to source_id.
df = (
    case.join(dept, 'dept_division')
    .join(source_df, 'source_id')
    .drop(dept.dept_division)
    .drop(source_df.source_id)
)
df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: long (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- num_hours_late: double (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- index: long (nullable = true)
 |-- source_username: string (nullable = true)



### 8. Are there any cases that do not have a request source?

In [26]:
# we can see there is no case using isnull
df.where(df.source_username.isNull()).count()

22/10/24 15:32:39 WARN TaskSetManager: Stage 24 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


0

### 9. What are the top 10 service request types in terms of number of requests?

In [27]:
#sql querry to check top 10 service request types by number request
spark.sql('''
SELECT
    service_request_type,
    COUNT(*) AS number_of_requests
FROM case
GROUP BY service_request_type
ORDER BY number_of_requests DESC
LIMIT 10;
''').show()

22/10/24 15:32:42 WARN TaskSetManager: Stage 29 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------------+
|service_request_type|number_of_requests|
+--------------------+------------------+
|           No Pickup|             86855|
|Overgrown Yard/Trash|             65895|
|        Bandit Signs|             32910|
|        Damaged Cart|             30338|
|Front Or Side Yar...|             28794|
|        Stray Animal|             26760|
|Aggressive Animal...|             24882|
|Cart Exchange Req...|             22024|
|Junk Vehicle On P...|             21473|
|     Pot Hole Repair|             20616|
+--------------------+------------------+



### 10. What are the top 10 service request types in terms of average days late?

In [28]:
#sql querry to check top 10 service request by average days late
spark.sql('''
SELECT
    service_request_type,
    AVG(num_days_late) AS avg_days_late
FROM case
GROUP BY service_request_type
ORDER BY avg_days_late DESC
LIMIT 10;
''').show()

22/10/24 15:32:45 WARN TaskSetManager: Stage 32 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------------+
|service_request_type|     avg_days_late|
+--------------------+------------------+
|Request for Resea...|               NaN|
|CCO_Request for R...|               NaN|
|  Zoning: Junk Yards|175.95636210420952|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...| 153.9972403942857|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|142.11255641500003|
|Zoning: Recycle Yard|135.92851612479797|
|Donation Containe...|131.75610506358706|
|License Requied U...|128.79828704142858|
+--------------------+------------------+



### 11. Does number of days late depend on department?

In [29]:
#function to check if number of days late depend on dept
#looks like it depends, others are late while others not
(
    df.groupBy('dept_name')
    .agg(F.mean('num_days_late').alias('avg_days_late'))
    .sort('avg_days_late', ascending = False)
    .show()
)

22/10/24 15:32:47 WARN TaskSetManager: Stage 35 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-------------------+
|           dept_name|      avg_days_late|
+--------------------+-------------------+
|        City Council|                NaN|
|                null| 135.92851612479794|
|    Customer Service|  59.73709149630046|
|Development Services| 13.433724555869698|
|Solid Waste Manag...|-2.2000575136721157|
|        Metro Health| -4.911766979607005|
|Parks and Recreation| -5.251521960055153|
|Trans & Cap Impro...|-20.612837354052697|
|Code Enforcement ...| -38.70133068329473|
|Animal Care Services| -226.5178394055047|
+--------------------+-------------------+



### 12. How do number of days late depend on department and request type?

In [30]:
#function to check if days late depend on dept & request type
(
    df.groupBy('dept_name', 'service_request_type')
    .agg(F.mean('num_days_late').alias('avg_days_late'))
    .sort('avg_days_late', ascending = False)
    .show()
)

22/10/24 15:32:52 WARN TaskSetManager: Stage 52 contains a task of very large size (18865 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+------------------+
|           dept_name|service_request_type|     avg_days_late|
+--------------------+--------------------+------------------+
|        City Council|Request for Resea...|               NaN|
|        City Council|CCO_Request for R...|               NaN|
|Code Enforcement ...|  Zoning: Junk Yards|175.95636210420932|
|Code Enforcement ...|Labeling for Used...|162.43032902285717|
|Code Enforcement ...|Record Keeping of...| 153.9972403942857|
|Code Enforcement ...|Signage Requied f...|151.63868055333333|
|Code Enforcement ...|Storage of Used M...|     142.112556415|
|                null|Zoning: Recycle Yard|135.92851612479794|
|Code Enforcement ...|Donation Containe...|131.75610506358709|
|Code Enforcement ...|License Requied U...|128.79828704142858|
|Trans & Cap Impro...|Traffic Signal Gr...|101.79846062200002|
|    Customer Service|           Complaint|  72.8705023031167|
|Code Enforcement ...|             Vendors| 66.54809898